1. Given a time series of E-mini S&P 500 futures, compute labels on one-
minute time bars using the fixed-horizon method, where τ is set at two
standard deviations of one-minute returns.

    1. Compute the overall distribution of the labels.
    2. Compute the distribution of labels across all days, for each hour of the 
    trading session.
    3. How different are the distributions in (b) relative to the distribution in (a)? Why ?

2. Repeat Exercise 1, where this time you label standardized returns 
(instead of raw returns), where the standardization is based on mean and variance
estimates from a lookback of one hour. Do you reach a different conclusion?

3. Repeat Exercise 1, where this time you apply the triple-barrier method on
volume bars. The maximum holding period is the average number of bars per
day, and the horizontal barriers are set at two standard deviations of bar
returns. How do results compare to the solutions from Exercises 1 and 2?